### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [1]:
!pip install openai cohere tiktoken -q

In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [2]:
import os
import openai
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["BOOTCAMP_OPENAI_KEY"]

### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [3]:
from openai import OpenAI

client = OpenAI()

In [4]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-9XAtsz0xiwp1dstVMduLoajGK1nV4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex are two completely different entities.\n\nLangChain is a project that aims to create a blockchain-based platform for language learning and teaching. It uses blockchain technology to facilitate language exchange, connect learners with tutors, and create a decentralized marketplace for language learning services.\n\nOn the other hand, LlamaIndex is a cryptocurrency index fund that tracks the performance of a basket of different cryptocurrencies. It allows investors to gain exposure to a diversified portfolio of cryptocurrencies without having to manage individual holdings.\n\nIn summary, LangChain is focused on language learning and education using blockchain technology, while LlamaIndex is a financial product that tracks the performance of cryptocurrencies.', role='assistant', function_call=None, tool_ca

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [5]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [6]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

LangChain and LlamaIndex are two completely different entities in the blockchain and cryptocurrency space.

LangChain is a blockchain platform that focuses on providing language-specific support for smart contracts and decentralized applications. It aims to bridge the language barrier that many developers face when trying to build applications on the blockchain by offering tools and resources in multiple languages.

On the other hand, LlamaIndex is a cryptocurrency market data provider that offers real-time prices, historical data, and other metrics for various cryptocurrencies and digital assets. It is designed to help traders and investors make informed decisions by providing them with accurate and reliable information about the cryptocurrency market.

In summary, LangChain is a blockchain platform for developing language-specific smart contracts, while LlamaIndex is a cryptocurrency market data provider for traders and investors.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [7]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I can't eat ice, and you're making me even more hungry by talking about it. How about we talk about something that will actually satisfy my hunger?

Let's try that same prompt again, but modify only our system prompt!

In [8]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I'm glad you're having an awesome day! As an AI, I don't have personal preferences, but both crushed ice and cubed ice have their own charm depending on how you want to enjoy them. Which type of ice do you prefer?

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [9]:
print(joyful_response)

ChatCompletion(id='chatcmpl-9XAw2xev4H4tJVHozTkL7FkwpPf95', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm glad you're having an awesome day! As an AI, I don't have personal preferences, but both crushed ice and cubed ice have their own charm depending on how you want to enjoy them. Which type of ice do you prefer?", role='assistant', function_call=None, tool_calls=None))], created=1717693886, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=30, total_tokens=80))


### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [10]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

Jane struggled to follow the stimple instructions for sewing a falbean costume for the school play.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [11]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

"I just used that stimple drill with the falbean attachment to assemble the furniture in no time."

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [12]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects because the total travel time for the flying and bus option is 5 hours, while the total travel time for the teleporter and bus option is only 1 hour. Therefore, if Billy wants to get home before 7PM EDT, he should choose the teleporter and bus option.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [13]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. 

If Billy chooses to fly and then take a bus, it will take him a total of 5 hours to get home (3 hours for the flight + 2 hours for the bus). Since it is currently 1PM local time, if Billy chooses this option, he will not make it home before 7PM EDT.

On the other hand, if Billy takes the teleporter and then a bus, it will take him a total of 1 hour to get home (0 hours for the teleporter + 1 hour for the bus). This option will get him home before 7PM EDT, which is his goal.

Therefore, Billy should choose to take the teleporter and then a bus in order to get home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

Now that you're accessing `gpt-3.5-turbo` through an API, developer style, let's move on to creating a simple application powered by `gpt-3.5-turbo`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)